In [2]:
#多分类逻辑回归
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import time
start = time.time()
#1.加载训练文件
data = pd.read_csv("/data/fjsdata/mathV.csv",sep='|',low_memory=False)#iterator=True)
#data = data.get_chunk(30000)
data = data.fillna(0)#填补空值为零
#data = data.dropna(axis=0,how='any')
print (data.shape)
#2.构造训练集和测试集
X_train, X_test, Y_train, Y_test = train_test_split(data.drop(columns=['quid','qdiff']), data['qdiff'], test_size=.2)#8成训练，2成测试
#3.lr多分类
clf = LogisticRegression(penalty='l2',random_state=0, solver='lbfgs', multi_class='multinomial').fit(X_train,Y_train)
#4.预测
Y_pred= clf.predict(X_test)
print ('测试集准确率：%f'% accuracy_score(Y_test, Y_pred))
end = time.time()
print ("Complete time: %f s" % (end - start))

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


(542340, 3586)
测试集准确率：0.838100
Complete time: 1635.191542 s


In [3]:
#贝叶斯多分类逻辑回归
import theano
import pymc3 as pm
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score
import theano.tensor as tt
from sklearn import datasets
import time
import numpy as np
starttime = time.time()
#1.加载训练文件
data = pd.read_csv("/data/fjsdata/mathV.csv",sep='|',low_memory=False)
data = data.fillna(0)#填补空值为零
print (data.shape)
#2.构造训练集和测试集
X_train, X_test, Y_train, Y_test = train_test_split(data.drop(columns=['quid','qdiff']), data['qdiff'], test_size=.2)#8成训练，2成测试
#3.构建softmax regression model
X_input = theano.shared(X_train.values)#转numpy array
Y_output = theano.shared(Y_train.values)#转numpy array
with pm.Model() as EDP_model:
    beta = pm.Normal('beta', mu=0, sd=6, shape=(3584,6))
    alpha = pm.Normal('alfa', mu=0, sd=6, shape=6)
    mu = tt.dot(X_input,beta) + alpha
    p = pm.Deterministic('p', tt.nnet.softmax(mu))
    Y_obs = pm.Categorical('Y_obs', p=p, observed=Y_output)
#4.nuts采样
with EDP_model:
    start=pm.find_MAP()  # 参数初猜
    step = pm.Metropolis()#多分类采样#step = pm.NUTS()连续采样
    trace = pm.sample(1000,start=start,step=step,chains=2,cores=2)
X_input.set_value(X_test.values)#测试集
Y_output.set_value(Y_test.values)#测试集
with EDP_model:
    ppc = pm.sample_posterior_predictive(trace)#后验预测 
    Y_pred = ppc['Y_obs'].mean(axis=0)
print ('测试集准确率：%f'% accuracy_score(Y_test,Y_pred.round()))
endtime = time.time()
print ("Complete time: %f s" % (endtime - starttime))

(542340, 3586)


/usr/local/lib/python3.6/dist-packages/pymc3/tuning/starting.py:61: UserWarning: find_MAP should not be used to initialize the NUTS sampler, simply call pymc3.sample() and it will automatically initialize NUTS in a better way.
  warnings.warn('find_MAP should not be used to initialize the NUTS sampler, simply call pymc3.sample() and it will automatically initialize NUTS in a better way.')
logp = -2.127e+05, ||grad|| = 11.838: 100%|██████████| 2756/2756 [12:04:03<00:00, 15.76s/it]   
Multiprocess sampling (2 chains in 2 jobs)
CompoundStep
>Metropolis: [alfa]
>Metropolis: [beta]
Sampling 2 chains: 100%|██████████| 3000/3000 [9:22:53<00:00, 20.43s/draws]  
The gelman-rubin statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.
100%|██████████| 2000/2000 [3:58:01<00:00,  7.19s/it]  


测试集准确率：0.835979
Complete time: 92672.074004 s


In [2]:
#决策树分类
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import time
start = time.time()
#1.加载训练文件
data = pd.read_csv("/data/fjsdata/mathV.csv",sep='|',low_memory=False)
data = data.fillna(0)#填补空值为零
#data = data.dropna(axis=0,how='any')
print (data.shape)
#2.构造训练集和测试集
X_train, X_test, Y_train, Y_test = train_test_split(data.drop(columns=['quid','qdiff']), data['qdiff'], test_size=.2)#8成训练，2成测试
#3.决策树分类
clf = DecisionTreeClassifier().fit(X_train,Y_train)
#4.预测
Y_pred= clf.predict(X_test)
print ('测试集准确率：%f'% accuracy_score(Y_test, Y_pred))
end = time.time()
print ("Complete time: %f s" % (end - start))

(30000, 3586)
测试集准确率：0.893500
Complete time: 360.567636 s


In [3]:
#LGBM集成学习分类
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score
import time
start = time.time()
#1.加载训练文件
data = pd.read_csv("/data/fjsdata/mathV.csv",sep='|',low_memory=False)
data = data.fillna(0)#填补空值为零
#data = data.dropna(axis=0,how='any')
print (data.shape)
#2.构造训练集和测试集
X_train, X_test, Y_train, Y_test = train_test_split(data.drop(columns=['quid','qdiff']), data['qdiff'], test_size=.2)#8成训练，2成测试
train_data=lgb.Dataset(X_train,label=Y_train)
validation_data=lgb.Dataset(X_test,label=Y_test)
#3.LGBM集成学习分类
params={'learning_rate':0.1,
        'lambda_l1':0.1,
        'lambda_l2':0.2,
        'max_depth':4,
        'objective':'multiclass',
        'num_class':6}
clf=lgb.train(params,train_data,valid_sets=[validation_data])
#4.预测
Y_pred= clf.predict(X_test)
Y_pred=[list(x).index(max(x)) for x in Y_pred]
print ('测试集准确率：%f'% accuracy_score(Y_test, Y_pred))
end = time.time()
print ("Complete time: %f s" % (end - start))

(30000, 3586)
[1]	valid_0's multi_logloss: 0.813482
[2]	valid_0's multi_logloss: 0.744894
[3]	valid_0's multi_logloss: 0.688876
[4]	valid_0's multi_logloss: 0.642017
[5]	valid_0's multi_logloss: 0.602135
[6]	valid_0's multi_logloss: 0.567829
[7]	valid_0's multi_logloss: 0.537986
[8]	valid_0's multi_logloss: 0.511875
[9]	valid_0's multi_logloss: 0.48889
[10]	valid_0's multi_logloss: 0.468556
[11]	valid_0's multi_logloss: 0.450494
[12]	valid_0's multi_logloss: 0.434406
[13]	valid_0's multi_logloss: 0.420025
[14]	valid_0's multi_logloss: 0.407099
[15]	valid_0's multi_logloss: 0.395517
[16]	valid_0's multi_logloss: 0.385069
[17]	valid_0's multi_logloss: 0.375629
[18]	valid_0's multi_logloss: 0.367126
[19]	valid_0's multi_logloss: 0.359422
[20]	valid_0's multi_logloss: 0.352438
[21]	valid_0's multi_logloss: 0.346092
[22]	valid_0's multi_logloss: 0.340335
[23]	valid_0's multi_logloss: 0.335088
[24]	valid_0's multi_logloss: 0.330293
[25]	valid_0's multi_logloss: 0.325944
[26]	valid_0's multi_

In [1]:
#贝叶斯神经网络
import theano
import pymc3 as pm
import sklearn
import numpy as np
from sklearn.cross_validation import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score
import time
import theano.tensor as tt
import tensorflow as tf
#1.贝叶斯神经网络模型定义
def construct_nn(ann_input, ann_output):
    n_hidden = 30
    
    # Initialize random weights between each layer
    init_1 = np.random.randn(3584, n_hidden).astype(float)#3584列特征
    init_2 = np.random.randn(n_hidden, n_hidden).astype(float)
    init_out = np.random.randn(n_hidden,6).astype(float)#6个类别
        
    with pm.Model() as neural_network:
        # Weights from input to hidden layer
        weights_in_1 = pm.Normal('w_in_1', 0, sd=1,  shape=(3584, n_hidden), testval=init_1)
        
        # Weights from 1st to 2nd layer
        weights_1_2 = pm.Normal('w_1_2', 0, sd=1,  shape=(n_hidden, n_hidden), testval=init_2)
        
        # Weights from hidden layer to output
        weights_2_out = pm.Normal('w_2_out', 0, sd=1,  shape=(n_hidden,6), testval=init_out)
        
        # Build neural-network using tanh activation function
        act_1 = pm.math.tanh(pm.math.dot(ann_input,weights_in_1))
        act_2 = pm.math.tanh(pm.math.dot(act_1, weights_1_2))
        act_out = pm.math.dot(act_2, weights_2_out)
        p = pm.Deterministic('p', tt.nnet.softmax(act_out))
        Y_obs = pm.Categorical('out', p=p, observed=ann_output)
    return neural_network
#2.加载训练文件
starttime = time.time()
data = pd.read_csv("/data/fjsdata/mathV.csv",sep='|',low_memory=False) 
data = data.fillna(0)#填补空值为零
print (data.shape)
#3.构造训练集和测试集
X_train, X_test, Y_train, Y_test = train_test_split(data.drop(columns=['quid','qdiff']), data['qdiff'], test_size=.2)#8成训练，2成测试
ann_input = theano.shared(X_train.values)
ann_output = theano.shared(Y_train.values)
#4.模型训练
neural_network = construct_nn(ann_input, ann_output) 
with neural_network:
    start=pm.find_MAP()  # 参数初猜
    step = pm.Metropolis()#多分类采样#step = pm.NUTS()连续采样
    trace = pm.sample(15000,start=start,step=step,chains=2,cores=2)
#5.后验预测
ann_input.set_value(X_test.values)#测试集
ann_output.set_value(Y_test.values)#测试集
with neural_network:
    ppc = pm.sample_posterior_predictive(trace)#后验预测 
    Y_pred = ppc['out'].mean(axis=0)
print ('测试集准确率：%f'% accuracy_score(Y_test,Y_pred.round()))
endtime = time.time()
print ("Complete time: %f s" % (endtime - starttime))

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


(30000, 3586)


/usr/local/lib/python3.6/dist-packages/pymc3/tuning/starting.py:61: UserWarning: find_MAP should not be used to initialize the NUTS sampler, simply call pymc3.sample() and it will automatically initialize NUTS in a better way.
  warnings.warn('find_MAP should not be used to initialize the NUTS sampler, simply call pymc3.sample() and it will automatically initialize NUTS in a better way.')
logp = -1.1357e+05, ||grad|| = 8.888: 100%|██████████| 2946/2946 [57:03<00:00,  1.16s/it]       
Multiprocess sampling (2 chains in 2 jobs)
CompoundStep
>Metropolis: [w_2_out]
>Metropolis: [w_1_2]
>Metropolis: [w_in_1]
Sampling 2 chains:  40%|████      | 12485/31000 [4:01:36<9:07:54,  1.78s/draws] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_win

测试集准确率：0.790000
Complete time: 78568.874756 s
